<a href="https://colab.research.google.com/github/shreyas-gowda02/CloudComputing/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip -q uninstall -y cudf-cu12 pylibcudf-cu12 pyarrow datasets

!pip -q install "pyarrow==21.0.0"

!pip -q install --no-deps "datasets==2.19.1" transformers accelerate evaluate scikit-learn gradio




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00


In [2]:
import pyarrow, datasets, torch, sys
print("Python:", sys.version.split()[0])
print("pyarrow:", pyarrow.__version__)
print("datasets:", datasets.__version__)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


ModuleNotFoundError: No module named 'pyarrow_hotfix'

In [5]:

%pip uninstall -y pyarrow datasets
%pip install -q "pyarrow==16.1.0" "datasets==3.0.1" "transformers==4.44.2" "accelerate==0.34.2" "evaluate==0.4.2" scikit-learn gradio


Found existing installation: pyarrow 16.1.0
Uninstalling pyarrow-16.1.0:
  Successfully uninstalled pyarrow-16.1.0
Found existing installation: datasets 3.0.1
Uninstalling datasets-3.0.1:
  Successfully uninstalled datasets-3.0.1


In [1]:
import pyarrow, datasets, transformers, torch
print("pyarrow:", pyarrow.__version__)
print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)
print("CUDA available:", torch.cuda.is_available())


pyarrow: 16.1.0
datasets: 3.0.1
transformers: 4.44.2
CUDA available: True


In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb, imdb["train"][0]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     unsupervised: Dataset({
         features: ['text', 'label'],
         num_rows: 50000
     })
 }),
 {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking

In [3]:
imdb


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
imdb["train"][0]


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
import pandas as pd
pd.DataFrame(imdb["train"][:5])


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [8]:
imdb["train"].features["label"]


ClassLabel(names=['neg', 'pos'], id=None)

In [9]:
from datasets import DatasetDict

train_all = imdb["train"].shuffle(seed=42)
imdb_splits = DatasetDict({
    "train":      train_all.select(range(20000)),
    "validation": train_all.select(range(20000, 25000)),
    "test":       imdb["test"]
})
imdb_splits


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [10]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score

X_train = imdb_splits["train"]["text"]
y_train = imdb_splits["train"]["label"]
X_val   = imdb_splits["validation"]["text"]
y_val   = imdb_splits["validation"]["label"]

tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1,2))
Xtr = tfidf.fit_transform(X_train)
Xva = tfidf.transform(X_val)

nb = MultinomialNB()
nb.fit(Xtr, y_train)

pred_val = nb.predict(Xva)
print("Baseline — Accuracy:", accuracy_score(y_val, pred_val))
print("Baseline — F1 (macro):", f1_score(y_val, pred_val, average="macro"))


Baseline — Accuracy: 0.8782
Baseline — F1 (macro): 0.8781985919757233


In [11]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized = imdb_splits.map(tokenize, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")
tokenized


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [15]:
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

args = TrainingArguments(
    output_dir="distilbert-imdb",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.293100,0.237683,0.907200
2,0.172900,0.278421,0.911000


TrainOutput(global_step=2500, training_loss=0.23846250228881835, metrics={'train_runtime': 972.6822, 'train_samples_per_second': 41.123, 'train_steps_per_second': 2.57, 'total_flos': 2649347973120000.0, 'train_loss': 0.23846250228881835, 'epoch': 2.0})

In [16]:
from sklearn.metrics import confusion_matrix, classification_report

test_metrics = trainer.evaluate(tokenized["test"])
print(test_metrics)

preds = trainer.predict(tokenized["test"])
y_true = preds.label_ids
y_pred = preds.predictions.argmax(axis=1)

print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred, digits=4))


test_texts = imdb["test"]["text"]
wrong_idx = np.where(y_true != y_pred)[0][:5]
for i in wrong_idx:
    print("\n---")
    print("Pred:", y_pred[i], " True:", y_true[i])
    print(test_texts[i][:400])


{'eval_loss': 0.268300861120224, 'eval_accuracy': 0.91008, 'eval_runtime': 183.8406, 'eval_samples_per_second': 135.987, 'eval_steps_per_second': 4.254, 'epoch': 2.0}
Confusion matrix:
 [[11246  1254]
 [  994 11506]]

Classification report:
               precision    recall  f1-score   support

           0     0.9188    0.8997    0.9091     12500
           1     0.9017    0.9205    0.9110     12500

    accuracy                         0.9101     25000
   macro avg     0.9103    0.9101    0.9101     25000
weighted avg     0.9103    0.9101    0.9101     25000


---
Pred: 1  True: 0
First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in th

---
Pre

In [18]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline


MODEL_DIR = "distilbert-imdb/best"

try:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
except Exception as e:
    print("Loading fine-tuned checkpoint failed:", e, "\nFalling back to base model.")
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

print(pipe("the movie was terrible"))


Loading fine-tuned checkpoint failed: distilbert-imdb/best is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>` 
Falling back to base model.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.50077885389328}]


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

import gradio as gr


id2label = getattr(model.config, "id2label", {0:"NEGATIVE", 1:"POSITIVE"})

def normalize_label(lbl):

    if isinstance(lbl, str) and lbl.startswith("LABEL_"):
        try:
            idx = int(lbl.split("_")[-1])
            return id2label.get(idx, lbl)
        except:
            return lbl
    return lbl

def predict_sentiment(text):
    try:
        out = pipe(text)[0]
        label = normalize_label(out["label"])
        score = float(out["score"])
        return f"{label} ({score:.3f})"
    except Exception as e:

        return f"Error: {e}"

demo = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, label="text"),
    outputs=gr.Textbox(label="output"),
    title="IMDb Sentiment Classifier"
)

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1ae298700a5965fd1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fbf0c9e12432f1e9af.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://1ae298700a5965fd1a.gradio.live
